In [13]:
import cv2 
import numpy as np
import time
import pickle
import pyrealsense2.pyrealsense2 as rs
sift_detector = cv2.SIFT_create()

In [ ]:
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
if depth_sensor.supports(rs.option.depth_units):
    depth_sensor.set_option(rs.option.depth_units,0.001)
depth_scale = depth_sensor.get_depth_scale()
align = rs.align(rs.stream.color)

In [ ]:
pipeline.stop()

In [ ]:
# world axis origin locate at the bottom ground of the camera, the depth camera is located (-30, 125, 0)
# camera: x: → y: ↓ z: o
# world: x: → y: ↑ z: x (using the cooridinates of vpython)
# need to rotate x: 180 deg, y: 0 deg, z: 0 deg, translate (-30, 125, 0) (mm)
camera_extrinsics = rs.extrinsics()
camera_extrinsics.rotation = [float(x) for x in [1, 0, 0, 0, np.cos(np.pi), -np.sin(np.pi), 0, np.sin(np.pi), np.cos(np.pi)]]
camera_extrinsics.translation = [-0.03, 0.125, 0.5] # specified in meter
bbox = [[240, 480], [719, 800]]
middle_pixel = ((240 + 719) // 2, (480 + 800) // 2)
pixels = [[middle_pixel[0] + (-2 + i) * 100, middle_pixel[1] + (-2 + i) * 100] for i in range(5)]
while True:
    frames = pipeline.wait_for_frames()
    if not frames:
        break

    aligned_frames = align.process(frames)   
    depth_frame = aligned_frames.get_depth_frame() 
    color_frame = aligned_frames.get_color_frame()
    aligned_intrinsic = color_frame.profile.as_video_stream_profile().intrinsics
    frame = np.asanyarray(color_frame.get_data())
    world_axis_points = []
    for pt in pixels:
        x, y = pt
        d = depth_frame.get_distance(y, x) # input y, x
        camera_axis_point = rs.rs2_deproject_pixel_to_point(aligned_intrinsic, (y, x), d) # input y, x
        world_axis_point = rs.rs2_transform_point_to_point(camera_extrinsics, camera_axis_point)
        cv2.circle(frame, (y, x), radius=5, color=(0, 255, 255), thickness=-1)
        world_axis_points.append(world_axis_point)

    print("                                         ".join(map(lambda x: str(["{:2f}".format(i*1000) for i in x]), world_axis_points)), end='\r')

    cv2.rectangle(frame, (bbox[0][1], bbox[0][0]), (bbox[1][1], bbox[1][0]), color=(0, 255, 0), thickness=2)
    cv2.circle(frame, (middle_pixel[1], middle_pixel[0]), radius=5, color=(0, 0, 255), thickness=-1)
    cv2.putText(frame, "Distance to middle box pixel: ({}, {}), World axis points of middle box pixel: ({:4f}, {:4f}, {:4f})".format(middle_pixel[0], middle_pixel[1], \
                        world_axis_point[0], world_axis_point[1], world_axis_point[2]),\
                org=(20, 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, thickness=2, fontScale=0.6, color=(0, 0, 0))
    cv2.imshow("Realsense RGB", frame)

    key = cv2.waitKey(1)
    if key == ord('s') or key == ord('S'):
        now = time.localtime(time.time())
        y = str(now.tm_year)
        m = str(now.tm_mon) if now.tm_mon > 9 else '0' + str(now.tm_mon)
        h = str(now.tm_hour) if now.tm_hour > 9 else '0' + str(now.tm_hour)
        u = str(now.tm_min) if now.tm_min > 9 else '0' + str(now.tm_min)
        s = str(now.tm_sec) if now.tm_sec > 9 else '0' + str(now.tm_sec)
        cv2.imwrite("{}{}{}{}{}_Kebuke.jpg".format(y, m, h, u, s), frame)
        
    if key == 27:  # ESC key: quit program
        cv2.destroyAllWindows()
        break

In [ ]:
#
    #
        #
            #
                #


['-107.581399', '-43.461908', '481.000036'],
['-54.944925', '8.628501', '477.000028'],
['-3.176244', '59.851333', '473.000020'],
['47.419373', '109.501652', '466.000021'],
# ['0.000000', '0.000000', '0.000000']

['-137.581393', '168.461904', '18.999964'],
['-84.944926', '116.371498', '22.999972'],
['-33.176243', '65.148667', '26.999980'],
['17.419374', '15.498348', '33.999979'],
# ['-29.999999', '125.000000', '500.000000']

In [283]:
# saving features [kpts:list, des:ndarray, relative point to cup origin:list]
fimg = cv2.imread("KEBUKE_feature/imgsrc/aging1.png")

# measured by ruler
fwidth_mm = 12 / 9 * 10
fheight_mm = 12 / 9 * 9.9
fx_mm = 0 # middle
fy_mm = 104 # middle
fz_mm = -7.3 # middle

fkpts_img, fdes_img = sift_detector.detectAndCompute(cv2.cvtColor(fimg, cv2.COLOR_BGR2GRAY), None)
fkpts_pts = []
f_relative_points = [] # world: x: → y: ↑ z: x
fh, fw, _ = fimg.shape
for fkpt in fkpts_img:
    x, y = fkpt.pt
    rx = fx_mm + fwidth_mm * (x - fw // 2) / fw
    ry = fy_mm + fheight_mm * (y - fw // 2) / fw
    rz = fz_mm * (y / (fh / 2))
    fkpts_pts.append([x, y])
    f_relative_points.append([rx, ry, rz])

save = [fkpts_pts, fdes_img, f_relative_points]
with open('KEBUKE_feature/aging1.pickle', 'wb') as f:
    pickle.dump(save, f)


In [301]:
for pt in fkpts_pts:
    cv2.circle(fimg, [int(x) for x in pt], 1, (0, 0, 255))
cv2.namedWindow("ouo", cv2.WINDOW_KEEPRATIO)
cv2.imshow("ouo", fimg)
cv2.waitKey()
cv2.destroyAllWindows()

In [302]:
cv2.imwrite("Feature_with_keypoints.jpg", fimg)

True

In [2]:
sift_detector = cv2.SIFT_create()
img = cv2.imread("KEBUKE/202306154114_Kebuke.jpg")
aging = cv2.imread("KEBUKE_feature/imgsrc/aging1.png")
kpts_img, des_img = sift_detector.detectAndCompute(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), None)
kpts_aging, des_aging = sift_detector.detectAndCompute(cv2.cvtColor(aging, cv2.COLOR_BGR2GRAY), None)

In [ ]:
kpts_bimg, des_bimg = sift_detector.detectAndCompute(cv2.GaussianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), ksize=(5, 5), sigmaX=0.7), None)
kpts_baging, des_baging = sift_detector.detectAndCompute(cv2.GaussianBlur(cv2.cvtColor(aging, cv2.COLOR_BGR2GRAY), ksize=(5, 5), sigmaX=0.7), None)

In [14]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)
img2 = img.copy()
bboxes = [[[425, 333], [655, 712]], [[360, 797], [719, 1019]]]
bkpts = None
bmatches = None
bmatches_mask = None
for bbox in bboxes:
    x1, y1 = bbox[0]
    x2, y2 = bbox[1]
    cv2.rectangle(img2, (y1, x1), (y2, x2), color=(0, 255, 0), thickness=2)
    kpts, des = sift_detector.detectAndCompute(cv2.cvtColor(img[x1:x2, y1:y2], cv2.COLOR_BGR2GRAY), None)
    for i in range(len(kpts)):
        y, x = kpts[i].pt
        kpts[i].pt = (y + y1, x + x1)

    matches = flann.knnMatch(des, des_aging, k=2)
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i]=[1,0]

    if isinstance(bkpts, tuple):
        for i in range(len(matches)):
            for j in range(2):
                matches[i][j].queryIdx += len(bkpts) 
        bkpts = tuple(list(bkpts) + list(kpts))
        bmatches = tuple(list(bmatches) + list(matches))
        bmatches_mask = bmatches_mask + matchesMask
    else:
        bkpts = kpts
        bmatches = matches
        bmatches_mask = matchesMask

draw_params = dict(matchColor = (0,255,0),
                singlePointColor = (255,0,0),
                matchesMask = bmatches_mask,
                flags = cv2.DrawMatchesFlags_DEFAULT)
img2 = cv2.drawMatchesKnn(img2,bkpts,aging,kpts_aging,bmatches,None,**draw_params)

cv2.namedWindow("ouo", cv2.WINDOW_KEEPRATIO)
cv2.imshow("ouo", img2)
cv2.waitKey()
cv2.destroyAllWindows()